In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

stl_path = "../processed_data/stl_energy_production_with_engineered_features.csv"
hp_path = "../processed_data/hp_energy_production_with_engineered_features.csv"

stl_df = pd.read_csv(stl_path)
hp_df = pd.read_csv(hp_path)

feature_cols = [
    "Water_Flow_m3_s", "avgtempC", "totalprecipMM", "humidity", "pressureMB",
    "WaterFlow_Diff_1d", "WaterFlow_Diff_7d", "WaterFlow_3day_avg", "WaterFlow_7day_avg",
    "Temp_Deviation", "WaterFlow_Humidity", "month_sin", "month_cos",
    "Normalized_Efficiency", "Prev_Day_Efficiency", "Prev_Week_Efficiency"
]
target_col = "Efficiency"

def prepare_data(df):
    X = df[feature_cols].values
    y = df[target_col].values.reshape(-1, 1)

    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    X = scaler_X.fit_transform(X)
    y = scaler_y.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
    y_train, y_test = torch.tensor(y_train, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)

    return X_train, X_test, y_train, y_test, scaler_y

stl_X_train, stl_X_test, stl_y_train, stl_y_test, stl_scaler_y = prepare_data(stl_df)
hp_X_train, hp_X_test, hp_y_train, hp_y_test, hp_scaler_y = prepare_data(hp_df)

batch_size = 64

stl_train_loader = DataLoader(TensorDataset(stl_X_train, stl_y_train), batch_size=batch_size, shuffle=True)
stl_test_loader = DataLoader(TensorDataset(stl_X_test, stl_y_test), batch_size=batch_size, shuffle=False)

hp_train_loader = DataLoader(TensorDataset(hp_X_train, hp_y_train), batch_size=batch_size, shuffle=True)
hp_test_loader = DataLoader(TensorDataset(hp_X_test, hp_y_test), batch_size=batch_size, shuffle=False)


In [18]:
class HydropowerEfficiencyNN(nn.Module):
    def __init__(self, input_dim):
        super(HydropowerEfficiencyNN, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(32, 1)  
        )

    def forward(self, x):
        return self.model(x)


input_dim = len(feature_cols)
model = HydropowerEfficiencyNN(input_dim)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

HydropowerEfficiencyNN(
  (model): Sequential(
    (0): Linear(in_features=16, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [19]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)

In [20]:
def train_model(model, train_loader, test_loader, num_epochs=50):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        scheduler.step()

        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")
            

train_model(model, stl_train_loader, stl_test_loader)

Epoch 5/50, Loss: 1.3649
Epoch 10/50, Loss: 1.2932
Epoch 15/50, Loss: 0.9217
Epoch 20/50, Loss: 0.5410
Epoch 25/50, Loss: 0.2673
Epoch 30/50, Loss: 0.4040
Epoch 35/50, Loss: 0.2449
Epoch 40/50, Loss: 0.1253
Epoch 45/50, Loss: 0.2786
Epoch 50/50, Loss: 0.1313


In [6]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate_model(model, test_loader, scaler_y):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_pred_batch = model(X_batch).cpu().numpy()
            y_true.append(y_batch.cpu().numpy())
            y_pred.append(y_pred_batch)

    y_true = np.concatenate(y_true, axis=0)
    y_pred = np.concatenate(y_pred, axis=0)

    y_true = scaler_y.inverse_transform(y_true)
    y_pred = scaler_y.inverse_transform(y_pred)

    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    print(f"Evaluation Results - R²: {r2:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}")

evaluate_model(model, stl_test_loader, stl_scaler_y)

Evaluation Results - R²: 0.9572, MAE: 0.0590, RMSE: 1.2721


# Some improvements made

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

stl_path = "../processed_data/stl_energy_production_with_engineered_features.csv"
hp_path = "../processed_data/hp_energy_production_with_engineered_features.csv"

stl_df = pd.read_csv(stl_path)
hp_df = pd.read_csv(hp_path)

feature_cols = [
    "Water_Flow_m3_s", "avgtempC", "totalprecipMM", "humidity", "pressureMB",
    "WaterFlow_Diff_1d", "WaterFlow_Diff_7d", "WaterFlow_3day_avg", "WaterFlow_7day_avg",
    "Temp_Deviation", "WaterFlow_Humidity", "month_sin", "month_cos",
    "Normalized_Efficiency", "Prev_Day_Efficiency", "Prev_Week_Efficiency"
]
target_col = "Efficiency"

def prepare_data(df):
    X = df[feature_cols].values
    y = df[target_col].values.reshape(-1, 1)

    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    X = scaler_X.fit_transform(X)
    y = scaler_y.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return (
        torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32),
        scaler_y
    )

stl_X_train, stl_X_test, stl_y_train, stl_y_test, stl_scaler_y = prepare_data(stl_df)
hp_X_train, hp_X_test, hp_y_train, hp_y_test, hp_scaler_y = prepare_data(hp_df)

batch_size = 64
stl_train_loader = DataLoader(TensorDataset(stl_X_train, stl_y_train), batch_size=batch_size, shuffle=True)
stl_test_loader = DataLoader(TensorDataset(stl_X_test, stl_y_test), batch_size=batch_size, shuffle=False)

hp_train_loader = DataLoader(TensorDataset(hp_X_train, hp_y_train), batch_size=batch_size, shuffle=True)
hp_test_loader = DataLoader(TensorDataset(hp_X_test, hp_y_test), batch_size=batch_size, shuffle=False)

In [13]:
class HydropowerEfficiencyNN(nn.Module):
    def __init__(self, input_dim):
        super(HydropowerEfficiencyNN, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.GELU(),  # 🔹 Use GELU instead of ReLU
            nn.Dropout(0.2),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.SiLU(),  # 🔹 Swish activation function
            nn.Dropout(0.2),

            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(32, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(16, 1)  # 🔹 Output Layer
        )

    def forward(self, x):
        return self.model(x)


input_dim = len(feature_cols)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = HydropowerEfficiencyNN(input_dim).to(device)

In [14]:
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)  

In [15]:
def train_model(model, train_loader, num_epochs=50):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        scheduler.step()

        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

train_model(model, stl_train_loader)

Epoch 5/50, Loss: 2.4254
Epoch 10/50, Loss: 1.4886
Epoch 15/50, Loss: 1.3925
Epoch 20/50, Loss: 1.1160
Epoch 25/50, Loss: 0.8589
Epoch 30/50, Loss: 0.6407
Epoch 35/50, Loss: 0.6818
Epoch 40/50, Loss: 0.5180
Epoch 45/50, Loss: 0.6824
Epoch 50/50, Loss: 0.2490


In [16]:
def evaluate_model(model, test_loader, scaler_y):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_pred_batch = model(X_batch).cpu().numpy()
            y_true.append(y_batch.cpu().numpy())
            y_pred.append(y_pred_batch)

    y_true = np.concatenate(y_true, axis=0)
    y_pred = np.concatenate(y_pred, axis=0)

    y_true = scaler_y.inverse_transform(y_true)
    y_pred = scaler_y.inverse_transform(y_pred)

    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    print(f"\nEvaluation Results - R²: {r2:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}")

evaluate_model(model, stl_test_loader, stl_scaler_y)


Evaluation Results - R²: 0.9912, MAE: 0.0568, RMSE: 0.5758


# RNN/LSTM/GRU
Best results after several tries

In [81]:
def create_sequences(df, feature_cols, target_col, seq_length=14):
    X, y = [], []
    for i in range(len(df) - seq_length):
        X.append(df[feature_cols].iloc[i:i+seq_length].values)  
        y.append(df[target_col].iloc[i+seq_length])  
    
    return np.array(X), np.array(y).reshape(-1, 1)

stl_X, stl_y = create_sequences(stl_df, feature_cols, target_col, seq_length=14)
hp_X, hp_y = create_sequences(hp_df, feature_cols, target_col, seq_length=14)


In [83]:
X_train, X_test, y_train, y_test = train_test_split(stl_X, stl_y, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test = map(
    lambda x: torch.tensor(x, dtype=torch.float32).to(device),
    (X_train, X_test, y_train, y_test)
)


batch_size = 64
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

Train shape: torch.Size([8028, 14, 16]), Test shape: torch.Size([2008, 14, 16])


In [84]:
class RecurrentModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=2, model_type="RNN", dropout_rate=0.3):
        super(RecurrentModel, self).__init__()
        self.model_type = model_type
        self.hidden_dim = hidden_dim
        
        if model_type == "RNN":
            self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout_rate)
        elif model_type == "LSTM":
            self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout_rate)
        elif model_type == "GRU":
            self.rnn = nn.GRU(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout_rate)
        
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.rnn(x)  
        return self.fc(out[:, -1, :])  


In [85]:
def train_model(model, train_loader, test_loader, num_epochs=75):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    criterion = nn.MSELoss()

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        scheduler.step(epoch_loss)  
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

    return model

In [86]:
def evaluate_model(model, test_loader, scaler_y):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)  
            y_pred_batch = model(X_batch).cpu().numpy()  
            y_true.append(y_batch.cpu().numpy())
            y_pred.append(y_pred_batch)

    y_true = np.concatenate(y_true, axis=0)
    y_pred = np.concatenate(y_pred, axis=0)

    y_true = scaler_y.inverse_transform(y_true)
    y_pred = scaler_y.inverse_transform(y_pred)

    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    print(f"Evaluation Results - R²: {r2:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}")
    return r2, mae, rmse


In [87]:
hidden_dim = 128 

for model_type in ["RNN", "LSTM", "GRU"]:
    print(f"\nTraining {model_type}...")
    model = RecurrentModel(len(feature_cols), hidden_dim, model_type=model_type)
    trained_model = train_model(model, train_loader, test_loader)
    r2, mae, rmse = evaluate_model(trained_model, test_loader, stl_scaler_y)
    print(f"{model_type} - R²: {r2:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}")


Training RNN...
Epoch 5/75, Loss: 12.7177
Epoch 10/75, Loss: 11.2720
Epoch 15/75, Loss: 10.2720
Epoch 20/75, Loss: 9.6341
Epoch 25/75, Loss: 9.2514
Epoch 30/75, Loss: 9.3310
Epoch 35/75, Loss: 9.0778
Epoch 40/75, Loss: 9.1271
Epoch 45/75, Loss: 9.1025
Epoch 50/75, Loss: 10.0806
Epoch 55/75, Loss: 9.3694
Epoch 60/75, Loss: 9.0561
Epoch 65/75, Loss: 9.0619
Epoch 70/75, Loss: 8.9237
Epoch 75/75, Loss: 8.8889
Evaluation Results - R²: 0.0525, MAE: 0.5077, RMSE: 16.5767
RNN - R²: 0.0525, MAE: 0.5077, RMSE: 16.5767

Training LSTM...
Epoch 5/75, Loss: 12.0572
Epoch 10/75, Loss: 9.7224
Epoch 15/75, Loss: 9.6891
Epoch 20/75, Loss: 9.4526
Epoch 25/75, Loss: 8.9858
Epoch 30/75, Loss: 8.9516
Epoch 35/75, Loss: 8.7706
Epoch 40/75, Loss: 8.7575
Epoch 45/75, Loss: 8.3188
Epoch 50/75, Loss: 8.0623
Epoch 55/75, Loss: 8.2088
Epoch 60/75, Loss: 8.5858
Epoch 65/75, Loss: 7.6321
Epoch 70/75, Loss: 7.6907
Epoch 75/75, Loss: 9.0339
Evaluation Results - R²: 0.0664, MAE: 0.5152, RMSE: 16.4548
LSTM - R²: 0.0664